In [ ]:
#in conmand prompt activate venv with venv\Scripts\activate


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re

In [ ]:
import faker
from faker import Faker

In [ ]:
# Set the seed
import random
random.seed(123)

In [ ]:
f = Faker(["en_UK"]) #british english version of faker

In [ ]:
#first dataset

#columns we want to try and generate
#offender_id
#offender_name
#breach_date = only between Jan to May 2023

#how many rows we want it to be
random_number = random.randint(15000, 20000)

In [ ]:
def generate_random_date(start_date, end_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    end_datetime = datetime.strptime(end_date, "%Y-%m-%d")

    # Calculate the total number of days between the start and end dates
    total_days = (end_datetime - start_datetime).days

    # Generate a random number of days to add to the start date
    random_days = random.randint(0, total_days)

    # Add the random number of days to the start date
    random_date = start_datetime + timedelta(days=random_days)

    return random_date.strftime("%Y-%m-%d")


In [ ]:
offender_id = []
offender_name = []
breach_date = []

#random_number
for i in range(1,random_number):
    
    #? means letters from the letters parameter, # means digits
    fake_off_id = f.bothify(text="???####???", letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    fake_name = f.name()
    fake_date = generate_random_date("2023-01-01", "2023-06-01")
    
    #print(i)
    offender_id.append(fake_off_id)
    offender_name.append(fake_name)
    breach_date.append(fake_date)

In [ ]:
#make into dataframe and save as csv
all_breaches_df = {
    "offender_id":offender_id,
    "offender_name":offender_name,
    "breach_date":breach_date
}

all_breaches_df = pd.DataFrame(all_breaches_df)

all_breaches_df

In [ ]:
#save that dataframe to current working directory
all_breaches_df.to_csv("all_breaches_df.csv", index=False)

In [ ]:
#Dataset 2

#offender_id
#tag_install_date
#tag_type
#age_at_tag_install
#sex
#ethnic_group
#original_offence


In [ ]:
#take a random number between 100-300 of the offender_id in all_breaches_df

rand_int_datatset_2 = random.randint(100, 300)

tag_breached_offenders = random.sample(offender_id, rand_int_datatset_2)

print(rand_int_datatset_2)

In [ ]:

breach_sample_on_tag = all_breaches_df.sample(frac=1, random_state=123)
# Sample the dataframe
breach_sample_on_tag = breach_sample_on_tag.head(rand_int_datatset_2)

#random.sample(all_breaches_df, rand_int_datatset_2)
breach_sample_on_tag = breach_sample_on_tag.drop('breach_date', axis=1)
breach_sample_on_tag.reset_index(drop=True, inplace=True)
breach_sample_on_tag

In [ ]:
#to get gender_guesser working it needs just the 1st name

#function to remove titles from 
def remove_titles(full_name):
    pattern = r'\b(Mr|Mrs|Ms|Dr|Miss|Master|Mister|Mrs.|Mr.|Dr.)\b'
    return re.sub(pattern, '', full_name).strip()

# Apply the function to the 'offender_name' column
breach_sample_on_tag['offender_name'] = breach_sample_on_tag['offender_name'].apply(remove_titles)

def extract_first_name(full_name):
    return full_name.split()[0]

breach_sample_on_tag['first_name'] = breach_sample_on_tag['offender_name'].apply(extract_first_name)
breach_sample_on_tag

In [ ]:
import gender_guesser.detector as gender

# Create a gender detector object
detector = gender.Detector(case_sensitive=False)

#it can't handle full names
#detector.get_gender("Bob McGee")
#detector.get_gender("Bob")

sex_types = ["male", "female"]
tag_types = ["curfew", "alcohol", "location"]
ethnic_groups = ["white", "black", "mixed", "asian", "other", "unknown"]
offence_groups = ["Violence against the person", "Theft offence", "Fraud", 
                  "Criminal Damage", "Drug offence", "Other Indictable",
                 "Summary motoring", "Summary non-motoring"]

In [ ]:
#make a separate df for all pilot tagged offenders

breached_offender_id_df_2 = breach_sample_on_tag["offender_id"]
breached_tag_install_date_df_2= []
breached_tag_type_df_2= []
breached_dob_df_2= []
breached_sex_df_2= []
breached_ethnic_group_df_2= []
breached_original_offence_df_2= []

#need to do separate fake generation for those who have already
#breached because we need to guess their gender
#ideally this generation of code would be functionalised to reduce code length
for i in range(0,len(breach_sample_on_tag)):
    
    fake_tag_install_date = generate_random_date("2022-08-01", "2022-12-25")
    fake_tag_type = random.sample(tag_types, 1)
    #need to do the str step otherwise the value is a list of length 1
    fake_tag_type = str(fake_tag_type[0])
    fake_dob = generate_random_date("1900-01-01", "2022-12-25")
    #guess sex at index
    fake_sex = detector.get_gender(breach_sample_on_tag["first_name"][i])
    fake_ethnic_group = random.sample(ethnic_groups, 1)
    fake_ethnic_group = str(fake_ethnic_group[0])
    fake_offence_group = random.sample(offence_groups, 1)
    fake_offence_group = str(fake_offence_group[0])

    breached_tag_install_date_df_2.append(fake_tag_install_date)
    breached_tag_type_df_2.append(fake_tag_type)
    breached_dob_df_2.append(fake_dob)
    breached_sex_df_2.append(fake_sex)
    breached_ethnic_group_df_2.append(fake_ethnic_group)
    breached_original_offence_df_2.append(fake_offence_group)

breached_tag_df = {
    "offender_id":breached_offender_id_df_2,
    "tag_install_date":breached_tag_install_date_df_2,
    "tag_type":breached_tag_type_df_2,
    "dob":breached_dob_df_2,
    "sex":breached_sex_df_2,
    "ethnic_group":breached_ethnic_group_df_2,
    "original_offence":breached_original_offence_df_2
}
breached_tag_df = pd.DataFrame(breached_tag_df)

unbreached_offender_id_df_2 = []
unbreached_tag_install_date_df_2= []
unbreached_tag_type_df_2= []
unbreached_dob_df_2= []
unbreached_sex_df_2= []
unbreached_ethnic_group_df_2= []
unbreached_original_offence_df_2= []


#want to make 1500- rand_int_datatset_2
for i in range(1, (1500-rand_int_datatset_2)+1):
    
    fake_offender_id = f.bothify(text="???####???", letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    fake_tag_install_date = generate_random_date("2022-08-01", "2022-12-25")
    fake_tag_type = random.sample(tag_types, 1)
    fake_tag_type = str(fake_tag_type[0])
    fake_dob = generate_random_date("1900-01-01", "2022-12-25")
    #guess sex at index
    fake_sex = random.sample(sex_types, 1)
    fake_sex = str(fake_sex[0])
    fake_ethnic_group = random.sample(ethnic_groups, 1)
    fake_ethnic_group = str(fake_ethnic_group[0])
    fake_offence_group = random.sample(offence_groups, 1)
    fake_offence_group = str(fake_offence_group[0])

    unbreached_offender_id_df_2.append(fake_offender_id)
    unbreached_tag_install_date_df_2.append(fake_tag_install_date)
    unbreached_tag_type_df_2.append(fake_tag_type)
    unbreached_dob_df_2.append(fake_dob)
    unbreached_sex_df_2.append(fake_sex)
    unbreached_ethnic_group_df_2.append(fake_ethnic_group)
    unbreached_original_offence_df_2.append(fake_offence_group)

unbreached_tag_df = {
    "offender_id":unbreached_offender_id_df_2,
    "tag_install_date":unbreached_tag_install_date_df_2,
    "tag_type":unbreached_tag_type_df_2,
    "dob":unbreached_dob_df_2,
    "sex":unbreached_sex_df_2,
    "ethnic_group":unbreached_ethnic_group_df_2,
    "original_offence":unbreached_original_offence_df_2
}
unbreached_tag_df = pd.DataFrame(unbreached_tag_df)

#join df of those who breached so far and the rest in the pilot
pilot_tag_df = pd.concat([breached_tag_df, unbreached_tag_df], ignore_index=True)


print(len(pilot_tag_df))# checking we have 1500 offender_ids


In [ ]:
pilot_tag_df

In [ ]:
pilot_tag_df.dtypes

In [ ]:
#fix some ambiguous results

pilot_tag_df.loc[pilot_tag_df["sex"]=="mostly_female","sex"] = "female"
pilot_tag_df.loc[pilot_tag_df["sex"]=="mostly_male","sex"] = "male"
pilot_tag_df.loc[pilot_tag_df["sex"]=="andy","sex"] = "unknown"

pilot_tag_df.sex.unique()


In [ ]:
#if we want, we can purposefully add some wrong results e.g. 50 wrong genders
#likely not needed right now - want students to focus more actual analysis than data issues

In [ ]:
#write pilot_tag_df as csv
pilot_tag_df.to_csv("pilot_tag_df.csv", index=False)